ANLP Assignment 2

19232697 - Saurabh Hebbalkar

# Neural Machine Translation - Assignment 2

In this task, you will develop a neural machine translation (NMT) system to translate text from one language into another. For this, you will need to choose the data to train the models, perform data processing and train a sequence2sequence neural model.


## Section 1- Data Collection and Preprocessing 


---


**Task 1  (5 marks)**

---

There are few datasets to train an NMT system available from Tatoeba Project (http://www.manythings.org/anki/) or OPUS project (http://opus.nlpl.eu/).

*  Download a language pair (preferably European language) and **extract** the file(s) and upload it to CoLab
*  Create a list of lines by splitting the text file at every occurrence accordingly, i.e. source and target language and remove copyright information
*  Print number of sentences
*  Limit the number of sentences to 10,000 lines (but more than 5,000 lines)
*  Split the data into train and test [You can split validation set here or while training use Keras validation_split option]
*  Print 100th sentence in original script[ not unicode] for source and target language

In [1]:
#English - German
#Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model
from numpy import array
from numpy import asarray
from numpy import zeros
import nltk
from nltk import bigrams, trigrams, ngrams
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Activation, dot, concatenate, TimeDistributed

Using TensorFlow backend.


In [2]:
#German File + Glove 100 file both are placed in Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Changing the directory on file to access the files
%cd drive/My\ Drive/ANLP

/content/drive/My Drive/ANLP


In [4]:
#Reading the English - German file 
filename="deu.txt"
with open(filename , 'r') as german_file:
  lines = german_file.read().split('\n')

result=[]
for x in lines:
    result.append(x.split('\t')[:2])

#Fetching only first 10000 Sentences
lines = result[:10000]
print("Total Number of Sentences:",len(lines))
print(lines[100])

Total Number of Sentences: 10000
['Beat it.', 'Mach ’ne Fliege!']


**Task 2 (5 marks)** 

---

* Add '<bof\>' to denote beginning of sentence and '<eos\>' to denote the end of the sentence to each target line.
* Preprocess (word tokenisation, lowercasing) the text.

In [0]:
#Defining the source & target list & set 
input_texts = list()
target_texts = list()
target_text_input = list()
source_words = set()
target_words = set()

In [0]:
num_samples = 10000
for line in lines[: min(num_samples, len(lines))]:
  
  #splitting the source and target text
  input_text, temp_tar_text = line[0], line[1]
  #Converting to lowercase
  input_text = input_text.lower()
  temp_tar_text = temp_tar_text.lower()
  #adding the start and end tag only to the target
  target_text = temp_tar_text + ' <eos>'
  target_text_input_temp = '<bof> ' + temp_tar_text
  #Appending it to list
  input_texts.append(input_text)
  target_texts.append(target_text)
  target_text_input.append(target_text_input_temp)

In [0]:
#Adding only the unique words to the dictionary for Source
for sentence in input_texts:
  words = sentence.split()
  for word in words:
    if word not in source_words:
      source_words.add(word)
#Adding only the unique words to the dictionary for Target
for sentence in target_texts:
  words = sentence.split()
  for word in words:
    if word not in target_words:
      target_words.add(word)

In [0]:
#Source & Target Vocabulary 
source_vocabulary = sorted(list(source_words))
target_vocabulary = sorted(list(target_words))
#Length of Source and Target Tokens
num_source_tokens = len(source_words)
num_target_tokens = len(target_words)+1
#Max size of source and target sentences
max_source_seq_length = max([len(txt) for txt in input_texts])
max_target_seq_length = max([len(txt) for txt in target_texts])

In [10]:
#Printing the results
print('Number of samples:', len(input_texts))
print('Number of unique source language tokens:', num_source_tokens)
print('Number of unique target language tokens:', num_target_tokens)
print('Max sequence length of source language:', max_source_seq_length)
print('Max sequence length of target language:', max_target_seq_length)
print("Source Vocabulary",source_vocabulary)
print("Target Vocabulary",target_vocabulary)

Number of samples: 10000
Number of unique source language tokens: 3085
Number of unique target language tokens: 4980
Max sequence length of source language: 15
Max sequence length of target language: 49
Source Vocabulary ['"aah."', '$10.00.', '$100.', '$30.', '$300.', '$5.', '12', '13', '17,', '18?', '19.', '2:00', '3.', '30?', '3:30.', '50', '65.', '7:45.', '8:00', '8:30.', '911.', '99%', '9:15.', 'a', 'a.', 'abandon', 'aboard!', 'aboard.', 'about', 'abroad.', 'absent.', 'absent?', 'absurd!', 'accelerated.', 'accept', 'accurate.', 'ache.', 'ached.', 'aches.', 'acne.', 'across.', 'act', 'action.', 'active.', 'actor.', 'add', 'addict.', 'addicted.', 'admirable.', 'admire', 'admired', 'admit', 'adopted', 'adopted.', 'adopted?', 'adore', 'adores', 'advice.', 'advises', 'afraid', 'afraid.', 'afraid?', 'after', 'afternoon.', 'again!', 'again.', 'against', 'age.', 'agree.', 'agree?', 'agreed.', 'agrees.', 'ahead!', 'ahead.', 'aim', 'aim.', "ain't", 'air.', 'alert.', 'alive.', 'alive?', 'all'

**Task 3 (5 marks)** 

---

---

*  Assign each unique word an integer value (5 marks).
*  Create word embedding for your vocabulary using pre-trained Glove embeddings (10 marks) (http://nlp.stanford.edu/data/glove.6B.zip)
* Print the first line of the embeddings (see below) 

In [11]:
#Dictonary of words, each word is converted into unique integer value for source and target language
input_token_index = dict([(word,i) for i, word in enumerate(source_words)])
target_token_index= dict([(word,i) for i, word in enumerate(target_words)])

print(len(input_token_index))
print(len(target_token_index))
# print(input_token_index)

3085
4979


In [0]:
#Embedding with Glove
embeddings_dictionary = dict()
#Getting Glove 100d file
glove_file = open(r'glove.6B.100d.txt', encoding="utf8")
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [0]:
# creating an embedding vector for each value in our unique
# source language.
#Setting Max num of words
MAX_NUM_WORDS = 10000
#Defining Embedding Shape
embedding_dim = embeddings_dictionary.get('a').shape[0]
num_words = min(MAX_NUM_WORDS, len(input_token_index))
embedding_matrix = zeros((num_source_tokens, embedding_dim))
for word, index in input_token_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        # If words are not found in embedding dictionary, it will be all-zeros
        embedding_matrix[index] = embedding_vector

In [0]:
#Printing First line of embedding
print(embedding_matrix[1])

[-0.60233003 -0.43066999 -0.050579   -0.51604998 -0.25003001  0.095017
 -0.51494002  0.80532998 -0.43268999  0.56735998  0.28883001 -0.50656003
  0.37138     0.12710001 -0.44266999 -1.02610004  0.96894997  0.16565999
 -0.91569     0.059401    0.89892    -0.43035001 -0.12737    -0.21546
  0.23238     0.11912    -0.27559    -0.37294     1.02279997 -0.030068
  0.042822   -0.14064001 -0.15496001 -0.58327001 -0.40650001  0.32898
 -0.33748999 -0.20298     0.43977001 -0.43292999  0.023072    0.068343
  1.04779994 -0.23901001  0.20482001  0.13564999 -0.54531002 -0.69310999
 -0.34939    -1.23870003  0.56384999  0.019323   -0.68619001  1.04330003
  0.077851   -0.79892999  0.26863    -1.27419996  2.36080003  0.42973
  0.16173001 -0.063217   -0.22765     0.096014    0.15469    -0.27911001
 -0.29409999 -0.39471999  0.24062     0.41396999 -0.12524    -0.69024003
  0.36071     0.29032001  0.39203     0.84381002  0.34588999  0.40762001
 -0.27689001 -0.43457001  0.14938     0.13569     0.28668001  0.77

## Section 2 Translation Model training



---



**Task 4 (15 marks)**
* Provide code for the encoder using Keras LSTM (5 marks)
* Provide code for the decoder using Keras LSTM (5 marks)
* Train the sequence2sequence (encoder-decoder) model (5 marks) 


In [0]:
#Tokenizing Input
input_tokenizer = Tokenizer(num_words= MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_texts)
input_integer_seq = input_tokenizer.texts_to_sequences(input_texts)
#Tokenizing Output
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(target_texts + target_text_input)
output_integer_seq = output_tokenizer.texts_to_sequences(target_texts)
output_input_integer_seq = output_tokenizer.texts_to_sequences(target_text_input)
#Vocab
source_vocabulary = input_tokenizer.word_index
target_vocabulary = output_tokenizer.word_index

#Padding the input sentences
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_source_seq_length)
#print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
#print("encoder_input_sequences[100]:", encoder_input_sequences[100])

#Padding the Decoder input (Post)
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_target_seq_length, padding='post')
#print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
#print("decoder_input_sequences[100]:", decoder_input_sequences[100])

#Padding the Decoder output
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_target_seq_length, padding='post')
#print("decoder_output_sequences.shape:", decoder_output_sequences.shape)
#print("decoder_output_sequences[100]:", decoder_output_sequences[100])

In [0]:
#Encoder code goes here
import numpy as np
encoder_input_data=np.zeros((len(input_texts),max_source_seq_length,num_source_tokens),dtype='float32')

In [0]:
# Decoder code goes here
decoder_input_data=np.zeros((len(input_texts), max_target_seq_length, num_target_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts), max_target_seq_length, num_target_tokens),dtype='float32')

In [17]:
#Printing the shape to check
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 15, 3085)
(10000, 49, 4980)
(10000, 49, 4980)


In [0]:
#One hot encoding the encoder sequence
for i, d in enumerate(encoder_input_sequences):
    for t, word in enumerate(d):
        encoder_input_data[i, t, word] = 1

In [0]:
#One hot encoding the decoder input sequence
for i, d in enumerate(decoder_input_sequences):
    for t, word in enumerate(d):
        decoder_input_data[i, t, word] = 1

In [0]:
#One hot encoding the decoder output sequence
for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_target_data[i, t, word] = 1

In [0]:
#Defining the LSTM Model
EMBEDDING_SIZE = 100
LSTM_NODES = 256
#Defining embedding layer for encoder
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_source_seq_length)

encoder_inputs_placeholder = Input(shape=(max_source_seq_length,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [0]:
#Decoder Placeholder
decoder_inputs_placeholder = Input(shape=(max_target_seq_length,))
#Embedding for decoder
decoder_embedding = Embedding(num_words, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)
#Decoder Output
decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [0]:
#Using Softmax activation
decoder_dense = Dense(num_target_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [24]:
#model= # your code goes here
model= Model([encoder_inputs_placeholder,decoder_inputs_placeholder], decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 49)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 15, 100)      308500      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 49, 256)      789760      input_2[0][0]                    
____________________________________________________________________________________________

In [25]:
#Fitting the model
 model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_target_data,
    epochs=10,
    validation_split=0.2
)
model.save('seq2seq_source_target.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 41s 5ms/step - loss: 0.5789 - accuracy: 0.9318 - val_loss: 0.4412 - val_accuracy: 0.9375
Epoch 2/10
8000/8000 [==============================] - 39s 5ms/step - loss: 0.3644 - accuracy: 0.9464 - val_loss: 0.4024 - val_accuracy: 0.9421
Epoch 3/10
8000/8000 [==============================] - 38s 5ms/step - loss: 0.3296 - accuracy: 0.9501 - val_loss: 0.3780 - val_accuracy: 0.9456
Epoch 4/10
8000/8000 [==============================] - 38s 5ms/step - loss: 0.3041 - accuracy: 0.9536 - val_loss: 0.3602 - val_accuracy: 0.9491
Epoch 5/10
8000/8000 [==============================] - 38s 5ms/step - loss: 0.2841 - accuracy: 0.9562 - val_loss: 0.3487 - val_accuracy: 0.9511
Epoch 6/10
8000/8000 [==============================] - 38s 5ms/step - loss: 0.2671 - accuracy: 0.9587 - val_loss: 0.3397 - val_accuracy: 0.9530
Epoch 7/10
8000/8000 [==============================] - 38s 5ms/step - loss: 0.252

## Section 3 Testing

---

**Task 5 (20 marks)**

* Use the trained model to translate the text from the source into the target language (10 marks). 
* Use the test/evaluation set (see Section 1) and perform an automatic evaluation with the BLEU metric (10 marks). 
You can use the NLTK library to calculate BLEU.

In [0]:
#Your code goes here
#Loading the model
model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('seq2seq_source_target.h5')

In [0]:
#Encoder model and decoder sates
encoder_model = Model(encoder_inputs_placeholder, encoder_states)
decoder_input_h = Input(shape=(256,))
decoder_input_c = Input(shape=(256,))
decoder_inputs_sates = [decoder_input_h, decoder_input_c]

In [0]:
#Decoder Inputs and states
decoder_inputs = Input(shape=(1,))
decoder_inputs_x = decoder_embedding(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_x, initial_state=decoder_inputs_sates)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
#Decoder Model
decoder_model = Model(
    [decoder_inputs] + decoder_inputs_sates,
    [decoder_outputs] + decoder_states
)

In [0]:
#Getting the index of words
idx2word_input = {v:k for k, v in source_vocabulary.items()}
idx2word_target = {v:k for k, v in target_vocabulary.items()}

In [0]:
#Translate Sentence
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_vocabulary['<bof>']
    eos = target_vocabulary['<eos>']
    output_sentence = []
 
    for _ in range(max_target_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])
 
        if eos == idx:
            break
 
        word = ''
 
        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)
 
        target_seq[0, 0] = idx
        states_value = [h, c]
 
    return ' '.join(output_sentence)

In [68]:
#Input sequence for encoder
seq_index = np.random.choice(len(input_texts))
input_seq = encoder_input_sequences[seq_index:seq_index+1]
decoded_sentence = translate_sentence(input_seq)
print('English Sentence:', input_texts[seq_index])
print('German Translation:', decoded_sentence)

English Sentence: tom knows why.
German Translation: tom kennt


In [0]:
#Importing Blue
import nltk
from nltk.translate.bleu_score import SmoothingFunction
smoothing = SmoothingFunction().method4

In [0]:
#Traing test split
from sklearn.model_selection import train_test_split
X_train , X_test , y_train, y_test = train_test_split(encoder_input_sequences, decoder_input_sequences , test_size=0.2)

In [0]:
#Bleu
def get_bleu(reference,hyp):
    try:
        return nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,smoothing_function = smoothing,weights=(0.35,0.35,0.20,0.10))
    except:
        print("Zero" , reference ,hyp)
        return 0

In [0]:
_inp = encoder_input_sequences.tolist()
X_test = X_test[:50]
bleu_scores = list()

In [0]:
#Generating Bleu Score
for _i,x in enumerate(X_test):
    idx = _inp.index(x.tolist())
    seq = encoder_input_sequences[idx:idx+1] 
    translation = translate_sentence(seq)
    reference = target_texts[idx].split()
    hypothesis = translation.split()
    reference = reference[:-1]
    _score = get_bleu(reference,hypothesis)
    bleu_scores.append( _score )

In [74]:
#Bleu Score Before Attention
_bleu_scores = np.array(bleu_scores)
print("Avg Score: "  ,_bleu_scores.mean() )

Avg Score:  0.3109140980759706


# Section 4 Attention

---



**Task 5 (40 Marks)** Sequence2Sequence

* Extend the existing Seq2Seq model with an attention mechanism [Discussed in Class]
* Create sequence2sequence model with attention (15 marks)
* Train the model with the same data from Section 1 (10 marks)
* Translate the evaluation set using the sequence2sequence attention model (10 marks)
* Evaluate the translations made with the sequence2sequence attention model and compare it with the model without attention using BLEU (5 marks)

In [0]:
#Creating an extended model
#Encoder
encoder_input = Input(shape=(max_source_seq_length,))
encoder = Embedding(num_words, EMBEDDING_SIZE, weights= [embedding_matrix], trainable=False)(encoder_input)
encoder = LSTM(256, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]
#Decoder
decoder_input = Input(shape=(max_target_seq_length,))
decoder = Embedding(num_target_tokens, EMBEDDING_SIZE)(decoder_input)
decoder = LSTM(256, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last,encoder_last])

In [0]:
# Attendtion model
attention = dot([decoder, encoder], axes=[2,2])
attention = Activation('softmax', name='attention')(attention)
context = dot([attention, encoder], axes=[2,1])
decoder_combined_context = concatenate([context, decoder])
output = TimeDistributed(Dense(64, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(num_target_tokens, activation="softmax"))(output)

In [77]:
#Attention Model Summary
model = Model(inputs =[encoder_input, decoder_input], outputs =[output])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 49)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 49, 100)      498000      input_7[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 15, 100)      308500      input_6[0][0]                    
____________________________________________________________________________________________

In [78]:
#Fitting the Attetion model
 model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_target_data,
    epochs=10,
    validation_split=0.2
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 20s 3ms/step - loss: 1.2274 - accuracy: 0.9196 - val_loss: 0.4733 - val_accuracy: 0.9313
Epoch 2/10
8000/8000 [==============================] - 16s 2ms/step - loss: 0.4028 - accuracy: 0.9413 - val_loss: 0.4441 - val_accuracy: 0.9388
Epoch 3/10
8000/8000 [==============================] - 16s 2ms/step - loss: 0.3802 - accuracy: 0.9435 - val_loss: 0.4395 - val_accuracy: 0.9386
Epoch 4/10
8000/8000 [==============================] - 16s 2ms/step - loss: 0.3674 - accuracy: 0.9450 - val_loss: 0.4350 - val_accuracy: 0.9398
Epoch 5/10
8000/8000 [==============================] - 16s 2ms/step - loss: 0.3565 - accuracy: 0.9461 - val_loss: 0.4160 - val_accuracy: 0.9412
Epoch 6/10
8000/8000 [==============================] - 16s 2ms/step - loss: 0.3465 - accuracy: 0.9470 - val_loss: 0.4147 - val_accuracy: 0.9410
Epoch 7/10
8000/8000 [==============================] - 16s 2ms/step - loss: 0.337

In [0]:
#Evaluation for the Attention Model
def translate_attention(seq):
    decoder_input = np.ones(shape=(seq.shape[0], max_target_seq_length))
  
    decoder_input[:,0] = target_vocabulary['<bof>']
    for i in range(1, max_target_seq_length):
        output_ = model.predict([seq, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output_[:,i]
    decoder_output = decoder_input[:,1:]
    decoder_output = np.array(decoder_output,dtype=int)
    sequence = decoder_output[0]
    text = ''
    for i in sequence:
        if i == 0:
            continue
        if idx2word_target[i] == '<eos>':
            break
        else:
            text += ' ' + idx2word_target[i]
    return text.strip().rstrip()

In [0]:
bleu_scores_att = list()

In [0]:
#Generating Bleu Score for attention 
for _i,x in enumerate(X_test):
    idx = _inp.index(x.tolist())
    seq = encoder_input_sequences[idx:idx+1] 
    translation = translate_attention(seq)
    reference = target_texts[idx].split()
    hypothesis = translation.split()
    reference = reference[:-1]
    _score = get_bleu(reference,hypothesis)
    bleu_scores_att.append( _score )


In [83]:
#Bleu Score After Attention
_bleu_scores_att = np.array(bleu_scores_att)
print("Avg Score After Attention: "  ,_bleu_scores.mean() )

Avg Score After Attention:  0.3109140980759706
